In [36]:
from stormvogel import *

In [203]:
%%prism grid_prism
mdp

const int N = 3; // Grid size

const int R_X = 0; // Ranger and lions initial positions
const int R_Y = N-1;
const int F_X = 0;
const int F_Y = 0;
const int C_X = N-1;
const int C_Y = (N-1)/2;


const int C1_X = 1; // Camera positions
const int C1_Y = 2;
const int C2_X = (N-1)/2;
const int C2_Y = 0;
const int C3_X = N-1;
const int C3_Y = N-2;

module ranger
    r_x : [0..N-1] init R_X;
    r_y : [0..N-1] init R_Y;
    
    c_1 : [0..2] init 0; // camera 1, 0: nothing, 1: fang, 2: claw
    c_2: [0..2] init 0;
    c_3: [0..2] init 0;
    i_c: [0..1] init 0; // inspected claw
    i_f: [0..1] init 0; // inspected fang

    // TODO add inspect action
    // TODO add action move for synchronization

    [left] (r_x > 0) ->      0.8 : (r_x' = r_x - 1) & (r_y' = r_y)
                           + 0.2 : (r_x' = r_x) & (r_y' = r_y);
    [right] (r_x < N-1) ->   0.8: (r_x' = r_x + 1) & (r_y' = r_y)
                           + 0.2 : (r_x' = r_x) & (r_y' = r_y);
    [up] (r_y > 0) ->        0.8: (r_x' = r_x) & (r_y' = r_y - 1)
                           + 0.2 : (r_x' = r_x) & (r_y' = r_y);
    [down] (r_y < N-1) ->    0.8: (r_x' = r_x) & (r_y' = r_y + 1)
                           + 0.2 : (r_x' = r_x) & (r_y' = r_y);
endmodule

module fang
    f_x : [0..N-1] init F_X;
    f_y : [0..N-1] init F_Y;

    [left] (fang_move_left = 1)  -> 0.03 : (f_x' = f_x - 1) & (f_y' = f_y);
    [left] (fang_move_right = 1) -> 0.03 : (f_x' = f_x + 1) & (f_y' = f_y);
    [left] (fang_move_up = 1) -> 0.03 : (f_x' = f_x) & (f_y' = f_y - 1);
    [left] (fang_move_down = 1) -> 0.03 : (f_x' = f_x) & (f_y' = f_y + 1);

    [right] (fang_move_left = 1)  -> 0.03 : (f_x' = f_x - 1) & (f_y' = f_y);
    [right] (fang_move_right = 1) -> 0.03 : (f_x' = f_x + 1) & (f_y' = f_y);
    [right] (fang_move_up = 1) -> 0.03 : (f_x' = f_x) & (f_y' = f_y - 1);
    [right] (fang_move_down = 1) -> 0.03 : (f_x' = f_x) & (f_y' = f_y + 1);

    [up] (fang_move_left = 1)  -> 0.03 : (f_x' = f_x - 1) & (f_y' = f_y);
    [up] (fang_move_right = 1) -> 0.03 : (f_x' = f_x + 1) & (f_y' = f_y);
    [up] (fang_move_up = 1) -> 0.03 : (f_x' = f_x) & (f_y' = f_y - 1);
    [up] (fang_move_down = 1) -> 0.03 : (f_x' = f_x) & (f_y' = f_y + 1);

    [down] (fang_move_left = 1)  -> 0.03 : (f_x' = f_x - 1) & (f_y' = f_y);
    [down] (fang_move_right = 1) -> 0.03 : (f_x' = f_x + 1) & (f_y' = f_y);
    [down] (fang_move_up = 1) -> 0.03 : (f_x' = f_x) & (f_y' = f_y - 1);
    [down] (fang_move_down = 1) -> 0.03 : (f_x' = f_x) & (f_y' = f_y + 1);
endmodule

module claw
    c_x : [0..N-1] init C_X; // Claw tribe
    c_y : [0..N-1] init C_Y;
endmodule

formula tribe_distance_x = f_x - c_x;
formula tribe_distance_y = f_y - c_y;

// Lion movement
formula lion_move_h1 = ((pow(pow(tribe_distance_y,2),0.5) > 1) | 
                        (tribe_distance_x > 2) | 
                        (tribe_distance_x > 1 & (pow(pow(tribe_distance_y,2),0.5) = 1)) |
                        (tribe_distance_x = 1 & (pow(pow(tribe_distance_y,2),0.5) > 1)));

formula lion_move_h2 = ((pow(pow(tribe_distance_y,2),0.5) > 1) | 
                       (tribe_distance_x < -2) | 
                       (tribe_distance_x < -1 & (pow(pow(tribe_distance_y,2),0.5) = 1)) |
                       (tribe_distance_x = -1 & (pow(pow(tribe_distance_y,2),0.5) > 1)));

formula lion_move_v1 = ((pow(pow(tribe_distance_x,2),0.5) > 1) | 
                        (tribe_distance_y > 2) | 
                        (tribe_distance_y > 1 & (pow(pow(tribe_distance_x,2),0.5) = 1)) |
                        (tribe_distance_y = 1 & (pow(pow(tribe_distance_x,2),0.5) > 1)));

formula lion_move_v2 = ((pow(pow(tribe_distance_x,2),0.5) > 1) | 
                        (tribe_distance_y < -2) | 
                        (tribe_distance_y < -1 & (pow(pow(tribe_distance_x,2),0.5) = 1)) |
                        (tribe_distance_y = -1 & (pow(pow(tribe_distance_x,2),0.5) > 1)));

// Fang movement possibilities
formula fang_move_left  = ((f_x > 0) & lion_move_h1) ? 1 : 0;
formula fang_move_right = ((f_x < 4) & lion_move_h2) ? 1 : 0;
formula fang_move_up    = ((f_y > 0) & lion_move_v1) ? 1 : 0;
formula fang_move_down  = ((f_y < 4) & lion_move_v2) ? 1 : 0;
formula fang_moves = fang_move_left + fang_move_right + fang_move_up + fang_move_down;

// Claw movement possibilities
formula claw_move_left  = ((c_y > 0) & lion_move_h2) ? 1 : 0;
formula claw_move_right = ((c_y < 4) & lion_move_h1) ? 1 : 0;
formula claw_move_up    = ((c_y > 0) & lion_move_v2) ? 1 : 0;
formula claw_move_down  = ((c_y < 4) & lion_move_v1) ? 1 : 0;
formula claw_moves = claw_move_left + claw_move_right + claw_move_up + claw_move_down;

In [208]:
from stormvogel import *
import stormpy
import stormvogel.stormpy_utils.mapping as mapping
model = stormpy.build_model(grid_prism)
print(model)

no_transitions = model.transition_matrix.nr_rows
if no_transitions < 100:
    sv_model = mapping.stormpy_to_stormvogel(model)
    show(sv_model)

-------------------------------------------------------------- 
Model type: 	MDP (sparse)
States: 	81
Transitions: 	1152
Choices: 	576
Reward Models:  none
State Labels: 	2 labels
   * deadlock -> 0 item(s)
   * init -> 1 item(s)
Choice Labels: 	none
-------------------------------------------------------------- 

